In [1]:
pip install synapseclient

     |████████████████████████████████| 213 kB 2.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
# test file
import synapseclient
import pandas as pd

SYN_LOGIN = 'jkobject'
SYN_PASSWORD = "QXAxTW2s4AcRjaz"
syn = synapseclient.Synapse() 
syn.login(SYN_LOGIN,SYN_PASSWORD) 

# Obtain a pointer and download the data 
# loading the DREAM Challenge MM RNAseq dataset from Synapse
# dataset analysed with Salmon (reference-free mapper),
# using hg19 gene annotation annotated with ENTREZ id + ERCC (likely ERCC92) which is the spike-in gene which are not present here
syn9744875 = syn.get(entity='syn9744875')
syn9744732 = syn.get(entity='syn9744732')
syn9926878 = syn.get(entity='syn9926878')

rna_data = pd.read_csv(syn9744875.path, sep='\t', index_col=0)
clinical_data = pd.read_csv(syn9926878.path, sep=',').set_index('Patient')
explanation = pd.read_csv(syn9744732.path, sep=',', index_col=0)

Welcome, jkobject!



In [93]:
len(set(mmrf_clinical['RNA-Seq']))

714

In [52]:
explanation.loc[set(clinical_data.columns) & set(explanation.index)]

,category,type,controlled_values,description,MMRF_IA8,GSE15695_UAMS,Hose,UAMS.UK.MyelomaXI,M2GEN,active
names,,,,,,,,,,
D_OS_FLAG,response,Numeric,"0=""No""; 1=""Yes""",Is the patient deceased?,STAND_ALONE_SURVIVAL.censos,NaN,OS_STATUS,OS_status,NaN,1.0
D_PFS,response,Numeric,NaN,Progression-free survival: number of days from...,STAND_ALONE_SURVIVAL.ttcpfs,NaN,as.integer(as.numeric(EFS) /12 * 365.25),PFS_months/12*365.25,NaN,1.0
D_Age,demographic,Numeric,NaN,Age at diagnosis (years),PER_PATIENT::D_PT_age,GSE15695_My9_GEP_clinical_data::age,age,Age,Demographics::AgeAtDiagnosis,1.0
D_Gender,demographic,Factor,Male; Female,Gender string,PER_PATIENT::DEMOG_GENDER,GSE15695_My9_GEP_clinical_data::sex,"recode(sex, ""male""=""Male"", ""female""=""Female"")","ifelse(grepl(""M"",toupper(df$Gender) ), ""Male"",...",Demographics::Gender,1.0
D_OS,response,Numeric,NaN,Overall Survival time: number of days from (tr...,STAND_ALONE_SURVIVAL.ttcos,NaN,as.numeric(OS) /12 * 365.25,OS_months/12*365.25,NaN,1.0
D_ISS,response,Factor,1; 2; 3,ISS Disease Stage,PER_PATIENT::D_PT_iss,GSE15695_My9_GEP_clinical_data::iss,ISS,"ISS column. `Stage I` = 1, `Stage II` = 2, `St...",NaN,1.0
Patient,id,Character,NaN,Normalized patient identification number.,MMRF_CoMMpass_IA8b_Seq_QC_Summary::Patients..K...,Array number,NaN,"format ""UAMS_0000""",Celgene Pt 1 = “1”,1.0
D_PFS_FLAG,response,Numeric,"0=""No""; 1=""Yes""",Has the patient developed progressive disease ...,STAND_ALONE_SURVIVAL.censpfs,NaN,EFS_STATUS,PFS_status,NaN,1.0


In [55]:
clinical_labels = {"D_OS": 'survival_days', "D_OS_FLAG": 'has_survived', "D_PFS": 'pfs_days', "D_PFS_FLAG": 'has_progressed', "HR_FLAG": 'to_pred'}
clinical_features = {'D_Age': 'age', 'D_Gender': 'sex', 'D_ISS': 'stage'}


In [71]:
clinical_data = clinical_data.rename(columns={**clinical_labels,**clinical_features})
# 
clinical_data = clinical_data[clinical_data.to_pred!='CENSORED']
# subsetting columns
clinical_data = clinical_data[list(clinical_features.values()) + list(clinical_labels.values())]

In [114]:
clinical_data

,age,sex,stage,survival_days,has_survived,pfs_days,has_progressed,to_pred
Patient,,,,,,,,
MMRF_1021,54,Female,1.0,1575,0,623,1,FALSE
MMRF_1029,46,Male,1.0,995,0,995,0,FALSE
MMRF_1030,65,Female,1.0,1572,0,1572,0,FALSE
MMRF_1031,62,Male,1.0,1581,0,1329,1,FALSE
MMRF_1032,76,Male,2.0,934,1,908,1,FALSE
...,...,...,...,...,...,...,...,...
MMRF_2535,46,Female,3.0,354,0,354,1,TRUE
MMRF_2554,65,Male,1.0,271,1,271,1,TRUE
MMRF_2557,83,Male,3.0,325,1,131,1,TRUE


In [33]:
from genepy.utils import helper as ghelp

In [34]:
gene_names = ghelp.generateGeneNames()

downloading gene names from biomart


In [38]:
gene_names = gene_names[gene_names.gene_biotype=='protein_coding'].set_index('ensembl_gene_id')

In [111]:
rna_data = rna_data.loc[set(gene_names.index) & set(rna_data.index)]

In [112]:
rna_data.index = [gene_names.loc[val]['hgnc_symbol'] for val in rna_data.index]

In [79]:
rna_data.columns

Index(['MMRF_2490_1_BM', 'MMRF_2499_1_BM', 'MMRF_1089_1_BM', 'MMRF_1778_1_BM',
       'MMRF_1777_1_BM', 'MMRF_1153_1_BM', 'MMRF_1392_1_BM', 'MMRF_1325_1_BM',
       'MMRF_1846_1_BM', 'MMRF_1935_1_BM',
       ...
       'MMRF_1847_1_BM', 'MMRF_1787_1_BM', 'MMRF_1048_1_BM', 'MMRF_2437_1_BM',
       'MMRF_1242_5_BM', 'MMRF_1497_1_BM', 'MMRF_1380_2_BM', 'MMRF_1700_1_BM',
       'MMRF_1918_1_BM', 'MMRF_1670_1_BM'],
      dtype='object', length=734)

In [95]:
len(set(rna_data.columns) - set(mmrf_clinical['RNA-Seq']))

81

In [98]:
# sorting to get the latest rnaseq available for a patient. By postulating that the number is equal to the timepoint at sampling 
col = list(rna_data.columns)
col.sort()
rna_data = rna_data[col]

In [100]:
rna_data.columns =  ['_'.join(i.split('_')[:2]) for i in rna_data.columns]

In [108]:
for val in ghelp.dups(rna_data.columns):
    loc = rna_data[val].iloc[:,-1]
    rna_data.drop(val, axis=1, inplace=True)
    rna_data[val] = loc

In [109]:
ghelp.dups(rna_data.columns)

[]

In [113]:
rna_data

,MMRF_1021,MMRF_1029,MMRF_1030,MMRF_1031,MMRF_1032,MMRF_1033,MMRF_1037,MMRF_1038,MMRF_1045,MMRF_1048,...,MMRF_2187,MMRF_1927,MMRF_1137,MMRF_1639,MMRF_1309,MMRF_1285,MMRF_1496,MMRF_1462,MMRF_2087,MMRF_1627
AC013470.2,0.172056,0.092393,0.084190,0.194376,0.056083,0.000000,0.000000,0.091177,0.000000,0.467040,...,0.102060,0.000000,0.000000,0.000000,0.664909,0.026829,0.045179,0.194342,0.000000,0.032598
WDR74,10.833300,18.580400,14.467400,18.063700,17.591600,24.509800,28.303400,9.711940,25.524200,43.862300,...,72.513000,21.131500,41.765900,20.392800,17.394100,32.254300,51.417000,96.478900,18.968400,20.637800
OPLAH,0.266774,1.296380,0.117927,0.117858,0.047435,1.354230,0.421337,0.095163,0.268473,1.420030,...,3.131060,0.087345,0.156914,0.165052,0.065000,0.074990,0.161000,2.192050,0.235179,0.121846
GOLM2,23.874900,24.682700,35.527700,51.381500,51.375400,21.487200,47.028400,46.390300,15.985900,36.320900,...,74.027000,25.138600,32.327900,37.428000,18.793500,54.014200,47.578000,34.092700,45.100500,19.999500
TMEM35A,0.004829,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.005257,0.000000,0.000000,0.004171,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MPDZ,0.714743,0.009763,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005945,0.013534,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041952,0.000000
PPP4R3C,0.000000,0.000000,0.051737,1.335520,0.009627,0.000000,0.004414,0.012250,0.005460,0.000000,...,0.174120,0.000000,3.336060,0.050069,0.000000,0.000000,0.000000,0.000000,0.000000,0.039021
SCML4,0.000000,0.010785,0.008280,0.000000,0.000000,0.011972,0.000000,0.000000,0.028731,0.064289,...,0.182476,0.117145,0.022507,0.000000,0.007208,0.008046,0.016038,0.000000,0.008534,0.026186
AC005520.1,0.050692,0.037980,0.054356,0.051392,0.019740,0.038427,0.000000,0.036468,0.077829,0.189108,...,0.064967,0.024270,0.022401,0.157403,0.021281,0.092562,0.042084,0.075233,0.107376,0.054280


In [125]:
len(clinical_data[clinical_data['stage'].isna()])

15

In [126]:
clinical_data = clinical_data[~clinical_data['stage'].isna()]

In [127]:
to_add = clinical_data[clinical_features.values()].replace({'Female': 1, 'Male': 0}).astype(int)

In [128]:
X = rna_data.T.loc[set(clinical_data.index)]

In [141]:
X = pd.concat([X, to_add], axis=1)

In [147]:
Y = clinical_data.replace({'FALSE':0, 'TRUE': 1})['to_pred'].loc[X.index].values

In [150]:
on = ['age', 'stage', 'sex', 'EMC92', 'UAMS70', 'PHF19', 'MMSET', 'BCL2A1']

In [159]:
X = X.astype(float)

In [171]:
data = pd.DataFrame(X.values, index=X.index, columns=X.columns)

In [151]:
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score, train_test_split

scaler = preprocessing.StandardScaler().fit(X[on].values)
X_scaled = scaler.transform(X.values)


clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X, Y)
scores = cross_val_score(clf, X, Y, cv=5)
print(scores.mean())

#clf = svm.SVC()

TypeError: 'Series' objects are mutable, thus they cannot be hashed